In [1]:
import api.daq as api
import os
import pickle
import time
import zmq

from datetime import datetime
from threading import Thread

In [2]:
help(api)

Help on module api.daq in api:

NAME
    api.daq

FUNCTIONS
    connected_febs(socket='tcp://localhost:5557')
        Returns a list containing the serial numbers of the connected febs
    
    load_config_file(path='CONF/CITIROC_SC_PROFILE1.txt', febs=[])
        Loads a configuration file for the given list of febs.
        If the list of febs is empty, load configuration for all the febs.
    
    set_thresholds(values, febs=[])
        Sets the threshold for the given febs.
        If only one threshold is set, set both thresholds to the same value.
        If the list of febs is empty, set the threshold for all the febs.
    
    set_voltages(values, febs=[])
        Sets the voltages for the given febs.
        If only one voltage is set, set all channels to the same value.
        If the list of febs is empty, set the voltages for all the febs.
    
    start_histos(febs=[], events=1000, driver='tcp://localhost:5555', input_socket='tcp://localhost:5556', output_socket='tcp://loc

### Program options

In [3]:
# These are the options or program arguments
# change these according to your needs
events   = 5000
driver   = 'tcp://localhost:5555'
data     = 'tcp://localhost:5556'
stats    = 'tcp://localhost:5557'
sink     = 'tcp://localhost:6000'
conf     = 'CONF/CITIROC_SC_PROFILE1.txt'

### Connect to driver and get list of connected FEBs

In [4]:
context = zmq.Context()
puller = context.socket(zmq.PULL)
puller.bind('tcp://*:6000')

In [5]:
febs = api.connected_febs(socket=stats)
api.load_config_file(path=conf, febs=febs)

### Data adquisition

In [ ]:
if str(input("Start measurements?")).lower()[0] == "y":
        
    path = str(input("where to save the data? [data/path/to/folder]: "))
    os.makedirs(path, exist_ok=True)
    
    while True:
    
        print("Started round at %s" % str(datetime.now()))
        histos = api.start_histos(
            febs=febs,
            events=events,
            driver=driver,
            input_socket=data,
            output_socket=sink,
            continuous=True
        )

        # Collect 10 histograms per module,
        # since the event rates differ some modules
        # will send more histograms than others
        # In order to know when a "round" is over
        # a list of counters is needed
        
        counters = [0]*len(febs)    
        while min(counters) < 10:
            task = puller.recv()
            mac5, config, pedestals, gains = api.task_to_data(task)
        
            # Count up the task
            counters[febs.index(mac5)] += 1

            now = str(datetime.now())
            print(now, " - got histograms from FEB %d" % mac5)
        
            f = open("%s/%02x-%s.task" % (path, mac5, now), "wb")
            pickle.dump(task, f)
            f.close()
            
            f = open("%s/%02x-%s.histos" % (path, mac5, now), "wb")
            pickle.dump((mac5, config, pedestals, gains), f)
            f.close()
    
        for hist in histos:
            hist.terminate()
    
        print("Finished round at %s" % str(datetime.now()))
        
        # Sleep for 60 seconds to allow to stop the observations
        time.sleep(60)